In [31]:
import pandas as pd
import joblib
import time

from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV  # Bayesian optimization: utilizado para optimizar hiperparámetros

import lightgbm as lgbm
from lightgbm import early_stopping  # Early stopping: utilizado para evitar sobreajuste

from Funcoes_Comuns import avaliar_modelo, registrar_modelo

Recuperar base já pré-processada

In [ ]:
# Obter dados
df_enem = pd.read_pickle('Bases\MICRODADOS_ENEM_2023_tratados.pkl')

# Base removendo as categorias que não são explicativas + as 10 variáveis menos importantes 
# df_enem = pd.read_pickle('Bases\MICRODADOS_ENEM_2023_tratados_variaveis_importante_explicativos.pkl')

In [48]:
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
# Ajuste de tipo para MLflow
# Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

In [50]:
categorical_features = X_train.select_dtypes(include=['category']).columns.tolist()

categorical_features

['CAT_COR_RACA',
 'CAT_CO_MUNICIPIO_ESC',
 'CAT_CO_UF_ESC',
 'CAT_DEPENDENCIA_ADM_ESC',
 'CAT_ENSINO',
 'CAT_ESCOLA',
 'CAT_ESTADO_CIVIL',
 'CAT_FAIXA_ETARIA',
 'CAT_LINGUA',
 'CAT_LOCALIZACAO_ESC',
 'CAT_NACIONALIDADE',
 'CAT_Q003',
 'CAT_Q004',
 'CAT_SEXO',
 'CAT_SIT_FUNC_ESC']

Modelo base

In [6]:
# Treinar modelo LGBMRegressor Base
modelo_lgbm = lgbm.LGBMRegressor(n_estimators=1000, 
                                 learning_rate=0.01, 
                                 random_state=42,
                                 max_bin=4095,
                                 force_row_wise=True)

start_time = time.time()

modelo_lgbm.fit(X_train, 
                y_train['NUM_NOTA_CH'], 
                eval_set=[(X_test, y_test['NUM_NOTA_CH'])], 
                eval_metric=['r2', 'rmse', 'mae'],
                categorical_feature=categorical_features)

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 4244
[LightGBM] [Info] Number of data points in the train set: 573256, number of used features: 40
[LightGBM] [Info] Start training from score 527.936960


In [7]:
# Previsões
y_pred = modelo_lgbm.predict(X_test)

In [8]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                 parametros={**modelo_lgbm.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                 X_train=X_train,
                 y_train=y_train,
                 y_test=y_test,
                 y_pred=y_pred,
                 variavel_alvo='NUM_NOTA_CH',
                 modelo=modelo_lgbm,
                 nome_modelo='modelo_lgbm_base',
                 descricao_modelo='Modelo LGBMRegressor base')

d:\Armazenamento\MBA\TCC\TccMbaUspEsalq\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'modelo_lgbm_base' already exists. Creating a new version of this model...
2025/05/21 00:03:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation.

🏃 View run carefree-fox-690 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/7b06c01669424b8aaa400e109a7ffae5
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_lgbm_base
Rastreamento do MLflow finalizado.


Created version '2' of model 'modelo_lgbm_base'.


In [9]:
modelo_lgbm.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.01,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 1000,
 'n_jobs': None,
 'num_leaves': 31,
 'objective': None,
 'random_state': 42,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'max_bin': 4095,
 'force_row_wise': True}

In [10]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred, "teste")

MAE (treino): 53.68
RMSE (treino): 68.10
R2 (treino): 0.35
MAE (teste): 55.34
RMSE (teste): 70.14
R2 (teste): 0.31


In [11]:
# Salvar o modelo otimizado como um arquivo pickle
joblib.dump(modelo_lgbm, 'Modelos\modelo_lgbm_base.pkl')

['Modelos\\modelo_lgbm_base.pkl']

Bayes Search

In [ ]:
modelo_lgbm_bayes = lgbm.LGBMRegressor(random_state=42,
                                       max_bin=4095, 
                                       force_row_wise=True)

In [53]:
# Definição do espaço de busca para otimização bayesiana
param_grid = {
    'num_leaves': (5, 60),                         # Número de folhas na árvore de decisão
    'max_depth': (40, 100),                        # Profundidade máxima da árvore
    'learning_rate': (0.005, 0.1, 'log-uniform'),  # Taxa de aprendizado
    'n_estimators': (5000, 6000),                  # Número de árvores
    'subsample': (0.3, 1.0),                       # Proporção de amostras usadas em cada árvore
    'colsample_bytree': (0.2, 1.0),                # Fração de colunas a serem usadas por árvore
    'reg_alpha': (1e-3, 1.0, 'log-uniform'),       # Regularização L1
    'reg_lambda': (1e-5, 1.0, 'log-uniform'),      # Regularização L2
}

In [54]:
# Configurar a busca Bayesiana usando BayesSearchCV

# Criando o otimizador Bayesiano
bayes_search = BayesSearchCV(
    estimator=modelo_lgbm_bayes,    # Modelo a ser otimizado
    search_spaces=param_grid,       # Espaço de busca definido acima
    scoring='r2',                   # Critério de seleção
    n_iter=30,                      # Número de avaliações do modelo
    cv=5,                           # Validação cruzada
    random_state=42,                # Semente para reprodutibilidade
    n_jobs=-1,                      # Paralelização total dos cálculos
    verbose=1                       # 0 = sem mensagens, 1 = mensagens de progresso, 2 = mensagens detalhadas
)

In [55]:
# Criar Eval Set para validação cruzada (15% do conjunto de treino)
X_train_bayes, X_eval, y_train_bayes, y_eval = train_test_split(
    X_train,
    y_train['NUM_NOTA_CH'],
    test_size=0.15,
    random_state=42
)

In [ ]:
fit_params = {
    'eval_set': [(X_eval, y_eval)],                     # Conjunto de validação
    'eval_metric': ['r2', 'rmse', 'mae'],               # Métricas a serem avaliadas
    'categorical_feature': categorical_features,        # Colunas categóricas
    'callbacks': [early_stopping(stopping_rounds=200)]  # Parar se não houver melhoria
}

In [41]:
# Executar a busca Bayesiana

start_time = time.time()
bayes_search.fit(X_train_bayes, y_train_bayes, **fit_params)

# Parar o cronômetro
end_time = time.time()
elapsed_time = end_time - start_time

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

In [ ]:
# Resultados da busca Bayesiana

print("Melhores parâmetros encontrados:")
print(bayes_search.best_params_)
print("R2: ", bayes_search.best_score_)
print(f"Tempo total de execução: {elapsed_time:.2f} segundos")

In [58]:
# Melhores parametros obtidos entre os modelos otimizados
parametros_best_bayes = {
    "colsample_bytree": 0.2297823151086026,
    "learning_rate": 0.028382425255806625,
    "max_depth": 40,
    "n_estimators": 5990,
    "num_leaves": 40,
    "reg_alpha": 0.00596084139054512,
    "reg_lambda": 0.8601375912019539,
    "subsample": 0.9434176158805601
}

In [59]:
# Treinar o modelo com os melhores parâmetros encontrados
modelo_lgbm_bayes.set_params(**bayes_search.best_params_)

start_time = time.time()

# Treinamento do modelo com os melhores parâmetros encontrados
modelo_lgbm_bayes.fit(X_train_bayes, 
                      y_train_bayes, 
                      eval_set=[(X_eval, y_eval)], 
                      eval_metric=['r2', 'rmse', 'mae'],
                      categorical_feature=categorical_features,
                      callbacks=[early_stopping(stopping_rounds=200)])

tempo_treino = time.time() - start_time

[LightGBM] [Warning] early_stopping_round is set=200, early_stopping_rounds=200 will be ignored. Current value: early_stopping_round=200
[LightGBM] [Info] Total Bins 4289
[LightGBM] [Info] Number of data points in the train set: 487267, number of used features: 40
[LightGBM] [Warning] early_stopping_round is set=200, early_stopping_rounds=200 will be ignored. Current value: early_stopping_round=200
[LightGBM] [Info] Start training from score 527.961360
Training until validation scores don't improve for 200 rounds
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[3452]	valid_0's rmse: 69.8276	valid_0's l1: 55.0658	valid_0's l2: 4875.9


In [60]:
# Previsões
y_pred_bayes = modelo_lgbm_bayes.predict(X_test)

In [61]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                    modelo=modelo_lgbm_bayes,
                    parametros={**modelo_lgbm_bayes.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                    X_train=X_train,
                    y_train=y_train,
                    y_test=y_test,
                    y_pred=y_pred_bayes,
                    variavel_alvo='NUM_NOTA_CH',
                    nome_modelo='modelo_lgbm_bayes',
                    descricao_modelo='Modelo LGBMRegressor otimizado com BayesSearchCV')

d:\Armazenamento\MBA\TCC\TccMbaUspEsalq\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'modelo_lgbm_bayes' already exists. Creating a new version of this model...
2025/06/02 22:30:54 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation

🏃 View run bittersweet-slug-181 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/5b3c10e72ca74e018ca4b6786632bcd3
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_lgbm_bayes
Rastreamento do MLflow finalizado.


Created version '4' of model 'modelo_lgbm_bayes'.


In [62]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm_bayes.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_bayes, "teste")

MAE (treino): 53.1950
RMSE (treino): 67.5541
R2 (treino): 0.3628
MAE (teste): 54.9856
RMSE (teste): 69.7555
R2 (teste): 0.3192


In [63]:
# Salvar o modelo otimizado como um arquivo pickle
joblib.dump(modelo_lgbm_bayes, 'Modelos\modelo_lgbm_bayes.pkl')

['Modelos\\modelo_lgbm_bayes.pkl']

### Buscar melhorar modelo alterando Features

In [ ]:
# Melhores parametros obtidos entre os modelos otimizados
parametros_best_bayes = {
    "colsample_bytree": 0.2297823151086026,
    "learning_rate": 0.028382425255806625,
    "max_depth": 40,
    "n_estimators": 5990,
    "num_leaves": 40,
    "reg_alpha": 0.00596084139054512,
    "reg_lambda": 0.8601375912019539,
    "subsample": 0.9434176158805601
}

### 1. Tentar melhorar removendo todas as variáveis sem categorias definidas ("Não informado")

    TP_ESTADO_CIVIL: 0 (Não informado)
    TP_COR_RACA: 0 (Não declarado)
    TP_NACIONALIDADE: 0 (Não informado)
    TP_ESCOLA: 1 (Não Respondeu)
    TP_ENSINO: 0 (Não informado)

In [3]:
# Obter dados
df_enem = pd.read_pickle('Bases\MICRODADOS_ENEM_2023_tratados_explicativos.pkl')

#Variaveis alvo
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajuste de tipo para MLflow -> Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

# Obter colunas categóricas
categorical_features = X_train.select_dtypes(include=['category']).columns.tolist()

# Criar Eval Set para validação cruzada (15% do conjunto de treino)
X_train, X_eval, y_train, y_eval = train_test_split(
    X_train,
    y_train,
    test_size=0.15,
    random_state=42
)

In [ ]:
modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas = lgbm.LGBMRegressor(random_state=42,
                                                                                max_bin=4095, 
                                                                                force_row_wise=True)

In [5]:
# Treinar o modelo com os melhores parâmetros encontrados
modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas.set_params(**parametros_best_bayes)

start_time = time.time()

# Treinamento do modelo com os melhores parâmetros encontrados
modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas.fit(X_train, 
                                                                y_train['NUM_NOTA_CH'], 
                                                                eval_set=[(X_eval, y_eval['NUM_NOTA_CH'])], 
                                                                eval_metric=['r2', 'rmse', 'mae'],
                                                                categorical_feature=categorical_features,
                                                                callbacks=[early_stopping(stopping_rounds=200)])

tempo_treino = time.time() - start_time

[LightGBM] [Warning] early_stopping_round is set=200, early_stopping_rounds=200 will be ignored. Current value: early_stopping_round=200
[LightGBM] [Info] Total Bins 4229
[LightGBM] [Info] Number of data points in the train set: 457253, number of used features: 40
[LightGBM] [Warning] early_stopping_round is set=200, early_stopping_rounds=200 will be ignored. Current value: early_stopping_round=200
[LightGBM] [Info] Start training from score 529.403061
Training until validation scores don't improve for 200 rounds
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[2697]	valid_0's rmse: 69.8743	valid_0's l1: 55.0455	valid_0's l2: 4882.41


In [6]:
# Previsões
y_pred_explicativas = modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas.predict(X_test)

In [7]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                    modelo=modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas,
                    parametros={
                                **modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas.get_params(), 
                                "amostra": X_train.shape[0], 
                                "tempo": tempo_treino
                                },
                    X_train=X_train,
                    y_train=y_train,
                    y_test=y_test,
                    y_pred=y_pred_explicativas,
                    variavel_alvo='NUM_NOTA_CH',
                    nome_modelo='modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas',
                    descricao_modelo='Modelo LGBMRegressor otimizado com BayesSearchCV sem categorias definidas')

d:\Armazenamento\MBA\TCC\TccMbaUspEsalq\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas' already exists. Creating a new version of this model...
2025/06/02 18:22:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seco

🏃 View run merciful-goat-238 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/70017590ea3a4de5862352707d4fe3ad
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas
Rastreamento do MLflow finalizado.


Created version '2' of model 'modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas'.


In [8]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_explicativas, "teste")

MAE (treino): 52.9660
RMSE (treino): 67.3000
R2 (treino): 0.3630
MAE (teste): 54.8429
RMSE (teste): 69.6780
R2 (teste): 0.3191


### 2. Tentar melhorar removendo variáveis de menor importância (importância e informação mútua)

    CAT_SIT_FUNC_ESC
    CAT_NACIONALIDADE
    CAT_ENSINO
    CAT_LOCALIZACAO_ESC
    CAT_ESTADO_CIVIL
    BIN_Q002_DUMMY_H
    NUM_Q017
    NUM_Q012
    BIN_Q001_DUMMY_H
    NUM_Q015

In [14]:
# Obter dados
df_enem = pd.read_pickle('Bases\MICRODADOS_ENEM_2023_tratados_variaveis_importantes.pkl')

#Variaveis alvo
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajuste de tipo para MLflow -> Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

# Obter colunas categóricas
categorical_features = X_train.select_dtypes(include=['category']).columns.tolist()

# Criar Eval Set para validação cruzada (15% do conjunto de treino)
X_train, X_eval, y_train, y_eval = train_test_split(
    X_train,
    y_train,
    test_size=0.15,
    random_state=42
)

In [ ]:
modelo_lgbm_reducao_de_features = lgbm.LGBMRegressor(random_state=42,
                                                        max_bin=4095, 
                                                        force_row_wise=True)

In [17]:
# Treinar o modelo com os melhores parâmetros encontrados
modelo_lgbm_reducao_de_features.set_params(**parametros_best_bayes)

start_time = time.time()

# Treinamento do modelo com os melhores parâmetros encontrados
modelo_lgbm_reducao_de_features.fit(X_train,
                                    y_train['NUM_NOTA_CH'], 
                                    eval_set=[(X_eval, y_eval['NUM_NOTA_CH'])], 
                                    eval_metric=['r2', 'rmse', 'mae'],
                                    categorical_feature=categorical_features,
                                    callbacks=[early_stopping(stopping_rounds=200)])

tempo_treino = time.time() - start_time

[LightGBM] [Warning] early_stopping_round is set=200, early_stopping_rounds=200 will be ignored. Current value: early_stopping_round=200
[LightGBM] [Info] Total Bins 4229
[LightGBM] [Info] Number of data points in the train set: 457253, number of used features: 40
[LightGBM] [Warning] early_stopping_round is set=200, early_stopping_rounds=200 will be ignored. Current value: early_stopping_round=200
[LightGBM] [Info] Start training from score 529.403061
Training until validation scores don't improve for 200 rounds
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[2697]	valid_0's rmse: 69.8743	valid_0's l1: 55.0455	valid_0's l2: 4882.41


In [18]:
# Previsões
y_pred_var_importantes = modelo_lgbm_reducao_de_features.predict(X_test)

In [19]:
nome_experimento = 'Notas CH ENEM 2023'

# Avaliar o modelo
registrar_modelo(experimento=nome_experimento,
                    modelo=modelo_lgbm_reducao_de_features,
                    parametros={
                                **modelo_lgbm_reducao_de_features.get_params(), 
                                "amostra": X_train.shape[0], 
                                "tempo": tempo_treino
                                },
                    X_train=X_train,
                    y_train=y_train,
                    y_test=y_test,
                    y_pred=y_pred_var_importantes,
                    variavel_alvo='NUM_NOTA_CH',
                    nome_modelo='modelo_lgbm_reducao_de_10_features',
                    descricao_modelo='Modelo LGBMRegressor otimizado com BayesSearchCV com redução de 10 features')

d:\Armazenamento\MBA\TCC\TccMbaUspEsalq\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Successfully registered model 'modelo_lgbm_reducao_de_10_features'.
2025/06/02 18:36:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_lgbm

🏃 View run ambitious-tern-384 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/06b5ee84eed94a2b809288e3b4b2054e
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_lgbm_reducao_de_10_features
Rastreamento do MLflow finalizado.


Created version '1' of model 'modelo_lgbm_reducao_de_10_features'.


In [20]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm_reducao_de_features.predict(X_train), "treino")
# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_var_importantes, "teste")

MAE (treino): 52.9660
RMSE (treino): 67.3000
R2 (treino): 0.3630
MAE (teste): 54.8429
RMSE (teste): 69.6780
R2 (teste): 0.3191


Conclusão

- Remoção de categorias não explicativas
    - mantendo fixo os parâmetros do modelo e removendo apenas as linhas com categorias não explicativas houve uma piora do modelo
    - R² de 0.3191 para 0.31908

- Remoção das piores 5/10 variáveis menos explicativas
    - mantendo fixo os parâmetros do modelo e removendo apenas as colunas com menor importância para o alvo houve uma piora do modelo
    - Removendo 05: R² de 0.3191 para 0.31908
    - Removendo 10: R² de 0.3191 para 0.31908

- Aplicando as duas ténicas
    - Redução R² de 0.3191 para 0.3144

Padrão se mantém para outras métricas RMSE e MAE